# Final Project - Data Cleaning

**Mathis Müller (19‑703‑081)**

**Jan‑Akim Albert Reimer (20‑488‑763)**

**Andrea Luca Taverna (20-710-372)**

**Luca Andrea Comino (20-703-641)**


# Setup

Before we start with the data cleaning, we import all the packages needed and setup the directory where our data can be found. In our case this is a Google Drive (**The examiner should change the directory accordingly**).


In [ ]:
!pip install geopandas
!pip install contextily

In [ ]:
from IPython.display import display_html, HTML
import os
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities, label_propagation_communities
import matplotlib.pyplot as plt
import itertools
import numpy as np
import random
import pandas as pd
from io import StringIO
from scipy.stats import poisson, expon
from scipy.special import factorial
import contextily as ctx
import geopandas as gpd
from matplotlib.patches import FancyArrowPatch
from matplotlib.cm import ScalarMappable, get_cmap
from matplotlib.colors import Normalize
from matplotlib.patches import Arc
import math


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Retrieve Dataset from Drive and manipulate it


First, we load the WTO-Dataset and get rid of misaligned columns. Then we restrict the dataset only on variables of interest and data-entries in the Product Code 8541 and 8542. Then we save the new dataset as a new csv-file.  

In [ ]:
#Consolidate columns & load into DataFrame
def consolidate_columns(input_file):
    with open(input_file, 'r', encoding='ISO-8859-1') as f:
        lines = f.readlines()
    # Replace ";" with a space or remove it entirely
    consolidated_lines = [''.join(line.strip().replace(';', '')) for line in lines]

    # Join all lines into a single CSV-like string
    consolidated_data = '\n'.join(consolidated_lines)

    # Use StringIO to simulate a file object for pandas
    data_io = StringIO(consolidated_data)

    # Read the cleaned data directly into a DataFrame
    df = pd.read_csv(data_io)
    return df

# Filter specific columns and restrict by Product/Sector Code
def filtered_dataframe(df):
    desired_columns = [
        "Reporting Economy Code",
        "Reporting Economy ISO3A Code",
        "Reporting Economy",
        "Partner Economy Code",
        "Partner Economy ISO3A Code",
        "Partner Economy",
        "Product/Sector Code",
        "Year",
        "Value"
    ]

    # Check for missing columns
    missing_columns = [col for col in desired_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"The following columns are missing from the data: {missing_columns}")

    # Filter only the desired columns
    filtered_df = df[desired_columns]

    # Convert 'Product/Sector Code' to string
    filtered_df["Product/Sector Code"] = filtered_df["Product/Sector Code"].astype(str)

    # Restrict to entries with 'Product/Sector Code' = '8541' or '8542'
    restricted_df = filtered_df[filtered_df["Product/Sector Code"].isin(["8541", "8542"])]

    return restricted_df

input_file = '/content/drive/MyDrive/ProjectWork/WtoData_20241125021407.csv'
output_file = '/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv'

# Consolidate and load directly into a DataFrame
consolidated_df = consolidate_columns(input_file)

# Filter specific columns and restrict by Product/Sector Code
try:
    restricted_df = filtered_dataframe(consolidated_df)
    # Save the final DataFrame to a CSV file
    restricted_df.to_csv(output_file, index=False)
    print(f"Filtered data saved to {output_file}")
except ValueError as e:
    print(e)

<ipython-input-26-d2f0bb28656f>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["Product/Sector Code"] = filtered_df["Product/Sector Code"].astype(str)


Filtered data saved to /content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv


## Clean the year 2023

Now that we have our consolidated the imporatant information for our network out of the raw data, we want to remove all the data for 2023.

We focus on the years 2017 to 2022, as these years contain sufficient data for meaningful insights. While trading data is available for 2023, it is incomplete compared to previous years. Notably, key countries such as China, USA, and Taiwan are missing from the 2023 dataset. Due to these gaps, we have decided to exclude 2023 data from our analysis.


In [ ]:
file_path = "/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv"
wto_consolidated_df = pd.read_csv(file_path)

# Filter out rows where "Year" is 2023
wto_consolidated_df = wto_consolidated_df[wto_consolidated_df["Year"] != 2023]

output_file = "/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv"
wto_consolidated_df.to_csv(output_file, index=False)

## Handling the European Union Data

We realized that many big countries are not reporting their imports, like Germany or France. Therefore we will now check which countries appear as trading partners ("Partner Economy"), but never appear as reporting economies.After running the analysis, we found that:  

- **90 out of 190 countries do not appear as "reporting economies"**. These are countries that are not WTO-members or are WTO-members but do not report imports for these specific product groups.

- We realised that **26 of the 27 EU member states are not reporting economies**, as the European Union often reports trade data collectively. This includes the United Kingdom (GBR) for the years 2017–2019, as it was still part of the EU during this period. To address this, we consolidate the trade volumes of all EU countries, including GBR for 2017–2019, into a single "European Union" (EEC) node. From 2020 onwards, GBR is treated as an independent country and is no longer consolidated into the EU node. This approach ensures the EU is accurately represented as a unified entity for the years 2017–2019, while reflecting the post-Brexit trade structure for 2020–2022.

- The other 64 countries we will ommit from our analysis. Most of them are small, geopolitically uninteresting countries. However, we also lose Russia and Iran, since these countries do not report their semiconductor imports.

In [ ]:
def load_csv(file_path):
    return pd.read_csv(file_path)

def analyze_iso3a_codes_from_csv(df):
    # Unique values
    reporting_iso3a_codes = df["Reporting Economy ISO3A Code"].unique()
    partner_iso3a_codes = df["Partner Economy ISO3A Code"].unique()

    print("Unique 'Reporting Economy ISO3A Code':")
    #print(reporting_iso3a_codes)
    print(len(reporting_iso3a_codes))

    print("\nUnique 'Partner Economy ISO3A Code':")
    #print(partner_iso3a_codes)
    print(len(partner_iso3a_codes))

    # Check differences
    missing_in_reporting = set(partner_iso3a_codes) - set(reporting_iso3a_codes)
    print("\nValues in 'Partner Economy ISO3A Code' not in 'Reporting Economy ISO3A Code':")
    print(missing_in_reporting)
    print(len(missing_in_reporting))

# File path to the CSV file
csv_file_path = '/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv'

# Load the DataFrame
df = load_csv(csv_file_path)

# Analyze the unique values and differences
analyze_iso3a_codes_from_csv(df)

Unique 'Reporting Economy ISO3A Code':
100

Unique 'Partner Economy ISO3A Code':
190

Values in 'Partner Economy ISO3A Code' not in 'Reporting Economy ISO3A Code':
{'IRQ', 'SSD', 'DMA', 'IRL', 'CZE', 'LBN', 'ROM', 'VEN', 'VNM', 'PNG', 'SLB', 'HUN', 'CUW', 'KEN', 'LTU', 'TJK', 'COG', 'MLT', 'ATG', 'KNA', 'LCA', 'UZB', 'FIN', 'SVN', 'DJI', 'IRN', 'LVA', 'FRA', 'BLR', 'LIE', 'BGR', 'DNK', 'CAF', 'LBY', 'SLE', 'DEU', 'RUS', 'NPL', 'SDN', 'TUN', 'LBR', 'BHS', 'YEM', 'ZMB', 'TCD', 'HRV', 'AGO', nan, 'SRB', 'ETH', 'KWT', 'VCT', 'ESP', 'AND', 'GNB', 'BIH', 'ITA', 'CYP', 'GMB', 'NLD', 'COD', 'BTN', 'GRC', 'EST', 'LUX', 'AFG', 'VUT', 'COM', 'SYR', 'STP', 'BRB', 'AUT', 'JAM', 'POL', 'DZA', 'GAB', 'MWI', 'VAT', 'PRT', 'BEL', 'GNQ', 'SWE', 'SOM', 'TLS', 'SVK', 'AZE', 'HTI', 'OMN', 'TKM', 'GRD'}
90


## Calculate weights for PSI EU

Before merging the European Union countries, we need to calculate the PSI value for the EU to support future analysis. To do this, we are determining the weight of each member country based on their relative export of semiconductor products. This approach ensures that smaller countries, like Ireland, which have significant semiconductor exports, are appropriately represented in terms of their impact on the stability of the EU's semiconductor trade. This method highlights the influence of export strength over sheer country size, providing a more suitable assessment of stability.



In [ ]:
file_path = '/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv'
data = pd.read_csv(file_path)

# Define EU countries for each year
eu_countries = {
    2017: ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland',
           'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
           'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'United Kingdom'],
    2018: ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland',
           'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
           'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'United Kingdom'],
    2019: ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland',
           'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
           'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'United Kingdom'],
    2020: ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland',
           'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
           'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden'],
    2021: ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland',
           'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
           'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden'],
    2022: ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland',
           'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
           'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']
}

# Normalize 'Partner Economy' for consistency
data['Partner Economy'] = data['Partner Economy'].str.strip().str.title()

# Ensure 'Value' is numeric and handle missing values
data['Value'] = pd.to_numeric(data['Value'], errors='coerce').fillna(0)

# Filter data for relevant years
data_filtered = data[data['Year'].between(2017, 2023)]

# Initialize an empty list to store results
weights_by_year = []

# Process each year
for year, countries in eu_countries.items():
    # Filter data for the specific year and relevant countries
    year_data = data_filtered[(data_filtered['Year'] == year) & (data_filtered['Partner Economy'].isin(countries))]

    # Group by Partner Economy and calculate total exports
    exports_by_country = year_data.groupby('Partner Economy')['Value'].sum().reset_index()

    # Calculate total exports for the year
    total_exports = exports_by_country['Value'].sum()

    # Avoid division by zero
    if total_exports > 0:
        # Calculate weights
        exports_by_country['Weight'] = exports_by_country['Value'] / total_exports
        # Add the year column for clarity
        exports_by_country['Year'] = year
        # Append results
        weights_by_year.append(exports_by_country)

# Combine all years into a single DataFrame
weights_df = pd.concat(weights_by_year, ignore_index=True)

# Load the original Political Stability Index Data
political_stability_file = '/content/drive/MyDrive/ProjectWork/Political_Stability_Data.csv'
political_stability_df = pd.read_csv(political_stability_file)

# Reshape the dataframe to long format
political_stability_long = political_stability_df.melt(
    id_vars=['Country Name', 'Country Code'],
    value_vars=[f'{year} [YR{year}]' for year in range(2017, 2023)],
    var_name='Year',
    value_name='PSI'
)

# Clean the year column
political_stability_long['Year'] = political_stability_long['Year'].str.extract(r'(\d{4})').astype(int)

# Merge weights into the reshaped political stability data based on Country Name and Year
political_stability_long = political_stability_long.merge(
    weights_df, left_on=['Country Name', 'Year'], right_on=['Partner Economy', 'Year'], how='left'
)

# Calculate weighted PSI for EU countries
weighted_psi_per_year = {}

for year in range(2017, 2023):
    # Filter for EU countries in the current year
    eu_countries_in_year = eu_countries[year]
    filtered_df = political_stability_long[
        (political_stability_long['Year'] == year) &
        (political_stability_long['Country Name'].isin(eu_countries_in_year))
    ]

    # Ensure numeric conversion for PSI and Weight
    filtered_df['PSI'] = pd.to_numeric(filtered_df['PSI'], errors='coerce')
    filtered_df['Weight'] = pd.to_numeric(filtered_df['Weight'], errors='coerce')

    # Calculate the weighted PSI
    total_weight = filtered_df['Weight'].sum()
    if total_weight > 0:
        weighted_psi_per_year[year] = (filtered_df['PSI'] * filtered_df['Weight']).sum() / total_weight
    else:
        weighted_psi_per_year[year] = None

# Add the EU's weighted PSI as a new row
eu_row_weighted = {
    'Country Name': 'European Union',
    'Country Code': 'ECC'
}
for year in range(2017, 2023):
    eu_row_weighted[f'{year} [YR{year}]'] = weighted_psi_per_year[year]

# Check if 'European Union' already exists in the dataset
if 'European Union' not in political_stability_df['Country Name'].values:
    eu_weighted_df = pd.DataFrame([eu_row_weighted])
    political_stability_df = pd.concat([political_stability_df, eu_weighted_df], ignore_index=True)

# Save the updated dataset
new_file_path_weighted = "/content/drive/MyDrive/ProjectWork/Political_Stability_Data.csv"
political_stability_df.to_csv(new_file_path_weighted, index=False)


<ipython-input-29-5439df3613c8>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PSI'] = pd.to_numeric(filtered_df['PSI'], errors='coerce')
<ipython-input-29-5439df3613c8>:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Weight'] = pd.to_numeric(filtered_df['Weight'], errors='coerce')


In the following code, we will we consolidate the trade volumes of all EU countries (with and without GBR), aswell as removing all the other countries that do not report the imports.

In [ ]:
def consolidate_eu_data_per_year_range(df, eu_countries, eu_countries_with_gbr, years_with_gbr):
    # List to store new rows for each subset
    new_rows = []

    # Process each year subset
    for year_range, eu_group in [(years_with_gbr, eu_countries_with_gbr), (list(range(2020, 2023)), eu_countries)]:
        # Filter data for the specific year range
        subset_df = df[df["Year"].isin(year_range)]

        # Iterate over each Reporting Economy
        for reporting_economy in subset_df["Reporting Economy"].unique():
            # Filter data for the current Reporting Economy
            reporting_df = subset_df[subset_df["Reporting Economy"] == reporting_economy]

            # Iterate over each unique Product/Sector Code for the current Reporting Economy
            for product_sector_code in reporting_df["Product/Sector Code"].unique():
                # Filter data for the current Product/Sector Code
                product_df = reporting_df[reporting_df["Product/Sector Code"] == product_sector_code]

                # Iterate over each unique Year for the current Product/Sector Code
                for year in product_df["Year"].unique():
                    # Filter data for the current Year
                    year_df = product_df[product_df["Year"] == year]

                    # Filter for rows where Partner Economy is in the current EU group
                    eu_rows = year_df[year_df["Partner Economy ISO3A Code"].isin(eu_group)]

                    # Sum the trade values for the EU countries
                    eu_value_sum = eu_rows["Value"].sum()

                    # Skip if there is no trade with the EU (sum is zero)
                    if eu_value_sum == 0:
                        continue

                    # Create a new row for the European Union (EEC)
                    new_row = {
                        "Reporting Economy": reporting_economy,
                        "Reporting Economy Code": year_df["Reporting Economy Code"].iloc[0],
                        "Reporting Economy ISO3A Code": year_df["Reporting Economy ISO3A Code"].iloc[0],
                        "Product/Sector Code": product_sector_code,
                        "Year": year,
                        "Partner Economy": "European Union",
                        "Partner Economy Code": "918",  # Fixed code for EU
                        "Partner Economy ISO3A Code": "EEC",  # Fixed code for EU
                        "Value": eu_value_sum
                    }

                    # Append the new row to the list
                    new_rows.append(new_row)

    # Remove the rows for the EU countries (including GBR for relevant years)
    df = df[
      ~(
          ((df["Partner Economy ISO3A Code"].isin(eu_countries)) & (df["Year"].isin(range(2017, 2023)))) |
          ((df["Partner Economy ISO3A Code"] == "GBR") & (df["Year"].isin(years_with_gbr)))
      )
    ]

    # Create a DataFrame from the new rows and append to the original DataFrame
    new_df = pd.DataFrame(new_rows)
    consolidated_df = pd.concat([df, new_df], ignore_index=True)

    return consolidated_df

# Define EU countries excluding GBR
eu_countries = [
    'FRA', 'BEL', 'DEU', 'BGR', 'SWE', 'IRL', 'CZE', 'POL', 'LUX', 'PRT',
    'FIN', 'LTU', 'MLT', 'GRC', 'EST', 'HRV', 'CYP', 'AUT', 'ESP', 'LVA',
    'NLD', 'ITA', 'DNK', 'SVN', 'HUN', 'SVK'
]

# Define EU countries including GBR (for 2017–2019)
eu_countries_with_gbr = eu_countries + ["GBR"]

# Define years when GBR was part of the EU
years_with_gbr = [2017, 2018, 2019]

# Apply the consolidation function
consolidated_df = consolidate_eu_data_per_year_range(wto_consolidated_df, eu_countries, eu_countries_with_gbr, years_with_gbr)


## Now we remove the other economies that do not report:

# Identify missing ISO3A codes
partner_iso3a_codes = set(consolidated_df["Partner Economy ISO3A Code"].unique())
reporting_iso3a_codes = set(consolidated_df["Reporting Economy ISO3A Code"].unique())
missing_in_reporting = partner_iso3a_codes - reporting_iso3a_codes

# Filter rows to remove
rows_to_remove = consolidated_df[
    (consolidated_df["Partner Economy ISO3A Code"].isin(missing_in_reporting)) &
    (~consolidated_df["Partner Economy ISO3A Code"].isin(eu_countries))
]

excluded_countries = rows_to_remove["Partner Economy"].unique()

# Drop rows from the DataFrame
cleaned_df = consolidated_df.drop(rows_to_remove.index)

# Save the consolidated DataFrame to a new file
output_file = '/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv'
cleaned_df.to_csv(output_file, index=False)

print(f"Consolidated data saved to {output_file}")

Consolidated data saved to /content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv


Now we create a list of the excluded countries as a LaTeX Tabel to use in our report.

In [ ]:
# Ensure an even split for the two columns
half = len(excluded_countries) // 2 + (len(excluded_countries) % 2)
column1 = sorted(excluded_countries[:half])
column2 = sorted(excluded_countries[half:])

# Pad the shorter column to ensure both are the same length
while len(column1) > len(column2):
    column2.append("")

# Create a DataFrame for the two columns
excluded_countries_df = pd.DataFrame({
    "Column 1": column1,
    "Column 2": column2
})

# Convert the DataFrame to LaTeX format
latex_table = excluded_countries_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{ll}
\toprule
Column 1 & Column 2 \\
\midrule
Afghanistan & Andorra \\
Algeria & Angola \\
Antigua and Barbuda & Bahamas \\
Azerbaijan & Barbados \\
Belarus & Bhutan \\
Bosnia and Herzegovina & Central African Republic \\
Chad & Comoros \\
Ethiopia & Congo \\
Haiti & Curaçao \\
Holy See & Democratic Republic of the Congo \\
Iran & Djibouti \\
Kenya & Dominica \\
Kuwait, the State of & Equatorial Guinea \\
Lebanese Republic & European Union, n.e.s. \\
Liberia & Gabon \\
Liechtenstein & Grenada \\
Malawi & Guinea-Bissau \\
Nepal & Iraq \\
Oman & Jamaica \\
Papua New Guinea & Libya \\
Romania & Saint Kitts and Nevis \\
Russian Federation & Saint Lucia \\
Saint Vincent and the Grenadines & Sao Tomé and Principe \\
Serbia & Solomon Islands \\
Sierra Leone & South Sudan \\
Somalia & Sudan \\
Tajikistan & Syrian Arab Republic \\
Timor-Leste & The Gambia \\
Tunisia & Turkmenistan \\
Venezuela, Bolivarian Republic of & Uzbekistan \\
Viet Nam & Vanuatu \\
World & Zambia \\
Yemen &

## Matching of stability index to countries in trade dataset

Unfortunatley, the country names in the PSI dataset do not match the country names in the WTO dataset. Also there are much more countries in the PSI dataset than in the WTO dataset:

In [ ]:
# Reload the two CSV files (paths assumed; update with actual paths if needed)
political_stability_file = "/content/drive/MyDrive/ProjectWork/Political_Stability_Data.csv"
wto_consolidated_file = "/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv"

# Read the data into pandas DataFrames
political_stability_df = pd.read_csv(political_stability_file)
wto_consolidated_df = pd.read_csv(wto_consolidated_file)

# Extract unique values from the specified columns
unique_country_names = political_stability_df["Country Name"].unique()
unique_reporting_economies = wto_consolidated_df["Reporting Economy"].unique()

# Convert to strings and filter out NaN or invalid entries
unique_country_names_list = sorted(
    str(name) for name in unique_country_names if pd.notnull(name)
)
unique_reporting_economies_list = sorted(
    str(name) for name in unique_reporting_economies if pd.notnull(name)
)

# Pad the shorter list with empty strings to ensure equal length
max_length = max(len(unique_country_names_list), len(unique_reporting_economies_list))
unique_country_names_list.extend([""] * (max_length - len(unique_country_names_list)))
unique_reporting_economies_list.extend([""] * (max_length - len(unique_reporting_economies_list)))


comparison_df = pd.DataFrame({
    "Country Names (Political Stability)": unique_country_names_list,
    "Reporting Economies (WTO)": unique_reporting_economies_list
})

# Ensure pandas displays all rows
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Find the countries in the Political Stability Index but not in the WTO Reporting Economies
countries_only_in_ps = set(unique_country_names_list) - set(unique_reporting_economies_list)

# Remove empty strings if they exist
countries_only_in_ps.discard("")

# Display the result
print("Countries in Political Stability Index but not in WTO Reporting Economies:")
print(sorted(countries_only_in_ps))

Countries in Political Stability Index but not in WTO Reporting Economies:
['Afghanistan', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Aruba', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Barbados', 'Belarus', 'Belgium', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Bulgaria', 'Cayman Islands', 'Central African Republic', 'Chad', 'Comoros', 'Congo Dem. Rep', 'Congo Rep', 'Cook Islands', "Cote d'Ivoire", 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Finland', 'France', 'French Guiana', 'Gabon', 'Gambia', 'Germany', 'Greece', 'Greenland', 'Grenada', 'Guam', 'Guinea-Bissau', 'Haiti', 'Hong Kong', 'Hungary', 'Iran', 'Iraq', 'Ireland', 'Italy', 'Jamaica', 'Jersey Channel Islands', 'Kenya', 'Kiribati', 'Korea Dem. People Rep.', 'Korea Rep', 'Kosovo', 'Kuwait', 'Lao PDR', 'Latvia', 'Lebanon', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Mac

With the help of chatGPT, we map the country names in the PSI dataset to the names in the WTO-dataset and merge the index in the index in the respective year and country to the WTO dataset.

We've merged the European Union (partner code) with the European Union as some countries like Switzerland, Ecuador, Mexcio or Pakistan report there imports form the EU under the name European Partner code

In [ ]:
political_stability_df = pd.read_csv(political_stability_file)
wto_df = pd.read_csv(wto_consolidated_file)

# County mapping for matching political stability index and WTO data set
# List of name differences
differences_list = [
    ("Bahrain", "Bahrain, Kingdom of"),
    ("Bolivia", "Bolivia, Plurinational State of"),
    ("Chinese Taipei", "Taiwan"),
    ("Cote d'Ivoire", "Côte d'Ivoire"),
    ("Hong Kong", "Hong Kong, China"),
    ("Korea Rep", "Korea, Republic of"),
    ("Lao PDR", "Lao People's Democratic Republic"),
    ("Macao China", "Macao, China"),
    ("Moldova", "Moldova, Republic of"),
    ("Saudi Arabia", "Saudi Arabia, Kingdom of"),
    ("Taiwan", "Chinese Taipei"),
    ("Turkiye", "Türkiye"),
    ("United States", "United States of America"),
    ("Viet Nam", "Vietnam")
]

# Create initial country mapping
country_mapping = {country: country for country in unique_reporting_economies}

# Add countries_only_in_ps with None values
country_mapping.update({country: None for country in countries_only_in_ps})

# Apply the name differences from differences_list
for old_name, new_name in differences_list:
    if old_name in country_mapping:
        country_mapping[old_name] = new_name

# Map Political Stability country names to WTO country names
political_stability_df["Mapped Country Name"] = political_stability_df["Country Name"].map(country_mapping)

# Replace "European Union (partner code)" in the WTO dataset
wto_df["Partner Economy"] = wto_df["Partner Economy"].replace("European Union (partner code)", "European Union")
wto_df["Reporting Economy"] = wto_df["Reporting Economy"].replace("European Union (partner code)", "European Union")

# Add new columns for PSI values
wto_df["PSI Reporting Economy"] = None
wto_df["PSI Partner Economy"] = None

# Iterate through WTO dataset to assign PSI values
for index, row in wto_df.iterrows():
    year = row["Year"]
    reporting_economy = row["Reporting Economy"]
    partner_economy = row["Partner Economy"]
    psi_column = f"{year} [YR{year}]"

    if psi_column in political_stability_df.columns:
        # Add PSI for Reporting Economy
        psi_value_reporting = political_stability_df.loc[
            political_stability_df["Mapped Country Name"] == reporting_economy, psi_column
        ]
        if not psi_value_reporting.empty:
            wto_df.at[index, "PSI Reporting Economy"] = psi_value_reporting.values[0]

        # Add PSI for Partner Economy
        psi_value_partner = political_stability_df.loc[
            political_stability_df["Mapped Country Name"] == partner_economy, psi_column
        ]
        if not psi_value_partner.empty:
            wto_df.at[index, "PSI Partner Economy"] = psi_value_partner.values[0]

output_file = "/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv"
wto_df.to_csv(output_file, index=False)

print(f"The updated dataset has been saved to {output_file}.")


The updated dataset has been saved to /content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv.


##Standardize Country Names

In the next step, we aim to establish a consistent naming convention for countries in our project. To achieve this, we will adopt the country names provided by GeoPandas, which will serve as the reference standard for the countries listed.

In [ ]:
# Load the WTO dataset
input_file = "/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv"
wto_df = pd.read_csv(input_file)

# Extract unique values for "Partner Economy" and "Reporting Economy"
unique_reporting_economies = wto_df["Reporting Economy"].unique()
unique_partner_economies = wto_df["Partner Economy"].unique()

# Display the unique values
print("Unique values in 'Reporting Economy':")
print(unique_reporting_economies)

print("\nUnique values in 'Partner Economy':")
print(unique_partner_economies)

Unique values in 'Reporting Economy':
['Albania' 'Argentina' 'Australia' 'Bahrain, Kingdom of' 'Bangladesh'
 'Armenia' 'Bolivia, Plurinational State of' 'Botswana' 'Brazil' 'Belize'
 'Brunei Darussalam' 'Myanmar' 'Burundi' 'Cambodia' 'Cameroon' 'Canada'
 'Cabo Verde' 'Sri Lanka' 'Chile' 'China' 'Chinese Taipei' 'Colombia'
 'Costa Rica' 'Cuba' 'Benin' 'Dominican Republic' 'Ecuador' 'El Salvador'
 'Fiji' 'Georgia' 'Ghana' 'Guatemala' 'Guinea' 'Guyana' 'Honduras'
 'Hong Kong, China' 'Iceland' 'India' 'Indonesia' 'Israel' "Côte d'Ivoire"
 'Japan' 'Kazakhstan' 'Jordan' 'Korea, Republic of' 'Kyrgyz Republic'
 "Lao People's Democratic Republic" 'Lesotho' 'Macao, China' 'Madagascar'
 'Malaysia' 'Maldives' 'Mali' 'Mauritania' 'Mauritius' 'Mexico' 'Mongolia'
 'Moldova, Republic of' 'Morocco' 'Mozambique' 'Namibia' 'New Zealand'
 'Nicaragua' 'Niger' 'Nigeria' 'Norway' 'Pakistan' 'Panama' 'Paraguay'
 'Peru' 'Philippines' 'Qatar' 'Rwanda' 'Saudi Arabia, Kingdom of'
 'Senegal' 'Seychelles' 'Singapor

In [ ]:
# Load the WTO dataset
input_file = "/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv"
wto_df = pd.read_csv(input_file)

# Load the dataset from the Natural Earth URL
url = "https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip"
world = gpd.read_file(url)

# Create a mapping dictionary
mapping = {
    'Bahrain, Kingdom of': 'Bahrain',
    'Bolivia, Plurinational State of': 'Bolivia',
    'Brunei Darussalam': 'Brunei',
    'Cabo Verde': 'Cape Verde',
    "Côte d'Ivoire": 'Ivory Coast',
    'Eswatini': 'Swaziland',
    'Hong Kong, China': 'Hong Kong',
    'Korea, Republic of': 'South Korea',
    "Lao People's Democratic Republic": 'Laos',
    'Macao, China': 'Macau',
    'Moldova, Republic of': 'Moldova',
    'Saudi Arabia, Kingdom of': 'Saudi Arabia',
    'Türkiye': 'Turkey',
    'United States of America': 'United States',
    'European Union (partner code)': 'European Union',
    'Chinese Taipei': 'Taiwan',
    'Macau': 'Macau',  # Redundant mapping for clarity
    'Samoa': 'Samoa',
    'Tanzania': 'Tanzania',
    'Bahrain': 'Bahrain',  # Redundant mapping for clarity
    'Mauritius': 'Mauritius',
    'Seychelles': 'Seychelles',
    'Swaziland': 'Swaziland',  # Redundant mapping for clarity
    'Cape Verde': 'Cape Verde',  # Redundant mapping for clarity
    'Maldives': 'Maldives',
    'Tonga': 'Tonga',
    'Kyrgyz Republic': 'Kyrgyzstan',
}

# Update the "Reporting Economy" and "Partner Economy" columns
wto_df["Reporting Economy"] = wto_df["Reporting Economy"].replace(mapping)
wto_df["Partner Economy"] = wto_df["Partner Economy"].replace(mapping)

# Verify that the names now match the GeoPandas standard names
unique_reporting = wto_df["Reporting Economy"].unique()
unique_partner = wto_df["Partner Economy"].unique()

# Save the updated dataset
output_file = "/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv"
wto_df.to_csv(output_file, index=False)

print(f"Updated dataset saved to {output_file}")

Updated dataset saved to /content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv


## Consolidate the single Headings



To accurately report the total trade in semiconductor goods, our network needs to consolidate the data under headings 8451 and 8542, where semiconductor products are categorized by the WTO. This involves grouping the data by partner and reporting economy. By merging these headings, we can determine the total imports of semiconductor goods of Reporting Country A from Partner Economy B for each year.

In [ ]:
# Load the consolidated CSV file
file_path = "/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv"
wto_consolidated_df = pd.read_csv(file_path)

# Define the years to analyze
filtered_years = wto_consolidated_df['Year'].unique()

# Initialize a list to store DataFrame fragments
dataframes = []

# Loop through each year
for year in filtered_years:
    # Filter rows for the specific year and product/sector codes 8541 and 8542
    filtered_df = wto_consolidated_df[
        (wto_consolidated_df['Year'] == year) &
        (wto_consolidated_df['Product/Sector Code'].isin([8541, 8542]))
    ]

    # Group by Reporting Economy, Partner Economy, PSI Reporting Economy, PSI Partner Economy, and Year
    year_sector_sum = filtered_df.groupby(
        ['Reporting Economy', 'Partner Economy', 'PSI Reporting Economy', 'PSI Partner Economy', 'Year']
    )['Value'].sum().reset_index()

    # Append the result to the list
    dataframes.append(year_sector_sum)

# Combine all yearly dataframes into one
final_dataframe = pd.concat(dataframes, ignore_index=True)

# Save the consolidated DataFrame to a CSV file
output_file = "/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv"
final_dataframe.to_csv(output_file, index=False)

## Constructing Trade Networks for Individual Years (2017–2022) and add the PSI to the Partner Economy (Exporting Country)

With our final dataset "WTO_consolidated_data" prepared, we will now create separate trade networks for each individual year from 2017 to 2022. This allows us to analyze the trade relationships and dynamics for each year independently. Futhermore we add the PSI for the exporting country which helps us to under stand how stable the exporting country is and therefore also the edge.

In [ ]:
# Load the consolidated CSV file
file_path = "/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv"
wto_consolidated_df = pd.read_csv(file_path)

# Create a directory to store the GML files
output_directory = "/content/drive/MyDrive/ProjectWork/WTO_trade_graphs_gml"
os.makedirs(output_directory, exist_ok=True)

# Filter data for years 2017 to 2022 and create graphs
filtered_years = [2017, 2018, 2019, 2020, 2021, 2022]

for year in filtered_years:
    # Filter rows for the specific year
    df_year = wto_consolidated_df[wto_consolidated_df['Year'] == year]

    # Create a directed graph for the year
    graph_trade = nx.DiGraph()

    # Collect all unique countries from Partner Economy (Exporting country)
    partner_countries = set(df_year['Partner Economy'])

    # Add all nodes (countries) with PSI attributes from Partner Economy
    for country in partner_countries:
        psi_partner = df_year.loc[df_year['Partner Economy'] == country, 'PSI Partner Economy'].mean()

        if not pd.isna(psi_partner):
            graph_trade.add_node(country, PSI=psi_partner)

    # Add edges with trade data
    for _, row in df_year.iterrows():
        reporting = row['Reporting Economy']
        partner = row['Partner Economy']
        value = row['Value']

        # Add edge with trade 'weight' (value)
        graph_trade.add_edge(partner, reporting, weight=value)

    # Save the graph as a GML file
    gml_file_path = os.path.join(output_directory, f"WTO_trade_graph_{year}.gml")
    nx.write_gml(graph_trade, gml_file_path)
    print(f"Graph for {year} saved as GML file at {gml_file_path}.")

print("All graphs have been successfully saved as GML files.")

Graph for 2017 saved as GML file at /content/drive/MyDrive/ProjectWork/WTO_trade_graphs_gml/WTO_trade_graph_2017.gml.
Graph for 2018 saved as GML file at /content/drive/MyDrive/ProjectWork/WTO_trade_graphs_gml/WTO_trade_graph_2018.gml.
Graph for 2019 saved as GML file at /content/drive/MyDrive/ProjectWork/WTO_trade_graphs_gml/WTO_trade_graph_2019.gml.
Graph for 2020 saved as GML file at /content/drive/MyDrive/ProjectWork/WTO_trade_graphs_gml/WTO_trade_graph_2020.gml.
Graph for 2021 saved as GML file at /content/drive/MyDrive/ProjectWork/WTO_trade_graphs_gml/WTO_trade_graph_2021.gml.
Graph for 2022 saved as GML file at /content/drive/MyDrive/ProjectWork/WTO_trade_graphs_gml/WTO_trade_graph_2022.gml.
All graphs have been successfully saved as GML files.


We created the network based on import data, where the **"Reporting" country** represents the country importing the semiconductor goods from the **"Partner" country**. By assigning the Partner country as the source (where the edge starts) and the Reporting country as the target (where the edge ends), the resulting graph effectively represents how much each country is exporting. Thus, we have constructed an export network with weighted edges which represent the value of exports.

## First Sanity Check

We want to see if the contents of the GML file are alline with our original data

In [ ]:
# Define the years to analyze
filtered_years = [2017, 2018, 2019, 2020, 2021, 2022]

# Initialize dictionaries to store Reporting and Partner Economies from the graphs
reporting_economies_by_year = {}
partner_economies_by_year = {}

# Define the directory where GML files are stored
gml_directory = "/content/drive/MyDrive/ProjectWork/WTO_trade_graphs_gml"

# Define the years to analyze
filtered_years = [2017, 2018, 2019, 2020, 2021, 2022]

# Dictionary to store results
graph_summary = {}

# Analyze each GML file
for year in filtered_years:
    gml_file_path = os.path.join(gml_directory, f"WTO_trade_graph_{year}.gml")
    try:
        # Load the graph
        graph_trade = nx.read_gml(gml_file_path)
        # Store the number of nodes and edges
        graph_summary[year] = {
            "Nodes": graph_trade.number_of_nodes(),
            "Edges": graph_trade.number_of_edges()
        }
        print(f"Year {year}: Nodes = {graph_trade.number_of_nodes()}, Edges = {graph_trade.number_of_edges()}")
    except FileNotFoundError:
        print(f"GML file for year {year} not found at {gml_file_path}.")

# Output the summary as a dictionary
graph_summary

# Define the years to analyze
filtered_years = [2017, 2018, 2019, 2020, 2021, 2022]

# Initialize dictionaries to track country appearances and missing years
country_counts = {}
country_missing_years = {}

# Loop through each year to process nodes from the graphs
for year in filtered_years:
    gml_file_path = os.path.join(gml_directory, f"WTO_trade_graph_{year}.gml")
    try:
        # Load the graph for the year
        graph_trade = nx.read_gml(gml_file_path)

        # Extract all countries from the nodes
        countries = {node for node in graph_trade.nodes}

        # Update counts and missing years
        for country in countries:
            if country not in country_counts:
                country_counts[country] = 0
                country_missing_years[country] = []
            country_counts[country] += 1

        # Check for countries missing in this year's graph
        for country in country_counts:
            if country not in countries:
                country_missing_years[country].append(year)

    except FileNotFoundError:
        print(f"GML file for year {year} not found at {gml_file_path}.")
        continue

# Summarize results
country_summary = {
    "Country Missing Years": {k: v for k, v in country_missing_years.items() if v}
}

country_summary

Year 2017: Nodes = 99, Edges = 2746
Year 2018: Nodes = 99, Edges = 2533
Year 2019: Nodes = 99, Edges = 2558
Year 2020: Nodes = 100, Edges = 2362
Year 2021: Nodes = 99, Edges = 2198
Year 2022: Nodes = 100, Edges = 2210


{'Country Missing Years': {'Burundi': [2021]}}

Burundi is the only country missing from the data for one year, specifically in 2021. In that year, the data should include 100 nodes. The addition of one node is attributed to Brexit, as the UK's departure from the EU resulted in changes to WTO statistics. From 2020 onwards, all countries began reporting imports from the UK separately, explaining the difference in node counts between 2019 and 2020.

No further action has been taken regarding Burundi's absence, as the country holds no significant role as a semiconductor exporter or importer. Consequently, its missing entry for that year does not introduce any bias into the dataset.



## First Validation of Network

In the network analysis part, we perform more detailed validation for our network. However, at this stage, our goal is to briefly ensure that the extraction process from the CSV file to the .GML file functions correctly. We achieve this by computing the differences between the data in the two files, randomly selecting various countries for comparison.

In [ ]:
# Validation of the data
# Load the consolidated CSV file
file_path = '/content/drive/MyDrive/ProjectWork/WTO_consolidated_data.csv'
wto_consolidated_df = pd.read_csv(file_path)

# Function to calculate the sum of values for specific countries, year, and product/sector codes
def get_sum_for_countries_and_year(reporting_country, partner_country, year):
    # Filter the DataFrame for the given criteria
    filtered_df = wto_consolidated_df[
        (wto_consolidated_df['Reporting Economy'] == reporting_country) &
        (wto_consolidated_df['Partner Economy'] == partner_country) &
        (wto_consolidated_df['Year'] == year)
            ]

    # Sum the 'Value' column
    total_value = filtered_df['Value'].sum()

    # Return the result
    return total_value

reporting_country = "Switzerland"
partner_country = "Canada"
year = 2018

# Get the sum from the CSV
csv_sum = get_sum_for_countries_and_year(reporting_country, partner_country, year)
print(f"The total sum of values for {partner_country} (Reporting) and {reporting_country} (Partner) in {year} is: {csv_sum}")

# Define the file path for the 2018 GML graph
gml_file_path = "/content/drive/MyDrive/ProjectWork/WTO_trade_graphs_gml/WTO_trade_graph_2018.gml"

# Load the graph and extract the edge weight
try:
    # Load the graph
    graph_trade_2018 = nx.read_gml(gml_file_path)

    # Check for an edge between Bangladesh and Albania
    edge_weight = graph_trade_2018.get_edge_data(partner_country, reporting_country, default=None)

    if edge_weight:
        gml_weight = edge_weight['weight']
        print(f"The edge weight between {partner_country} and {reporting_country} in 2018 is: {gml_weight}")
    else:
        print(f"No edge found between {partner_country} and {reporting_country} in 2018.")
        gml_weight = 0  # Assume 0 if no edge exists

except FileNotFoundError:
    print(f"GML file for 2018 not found at {gml_file_path}.")
    gml_weight = None
except Exception as e:
    print(f"An error occurred: {e}")
    gml_weight = None

# Calculate the difference
if gml_weight is not None:
    difference = csv_sum - gml_weight
    print(f"The difference between the CSV value and GML edge weight is: {difference}")
else:
    print("Unable to calculate the difference due to missing GML data.")

The total sum of values for Canada (Reporting) and Switzerland (Partner) in 2018 is: 3603205.9051365
The edge weight between Canada and Switzerland in 2018 is: 3603205.9051365
The difference between the CSV value and GML edge weight is: 0.0
